Probability density curves are in "**Distributions**"

Run this to create testcases, mutants and distributions of your own

#### **NOTE:** 
* If creating new testcases and their mutants, make sure to name the new testcases differently from the already existing testcases.
* In _muskit\functionalities.py_ line 160, change the variable *_venv\_python_* to the path of your virtual environment containing the qiskit modules.


In [ ]:
import random
import math
import os
import subprocess
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [2]:

def generate_random_qiskit_circuit(num_qubits, num_operations, filename):
    code = f"""import math
from qiskit import *

q = QuantumRegister({num_qubits}, 'q')
c = ClassicalRegister({num_qubits}, 'c')
qc = QuantumCircuit(q, c)

"""
    
    gates = ["x", "h", "p", "t", "s", "z", "y", "id", "rx", "ry", "rz", "sx", "swap", "rzz", "rxx", "cx", "cz", "ccx", "cswap"]
    
    for _ in range(num_operations):
        gate = random.choice(gates)
        
        if gate in ['x', 'h', 't', 's', 'z', 'y', 'id', 'sx']:
            qubit = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{qubit}])\n"
        elif gate in ['p', 'rx', 'ry', 'rz']:
            qubit = random.randint(0, num_qubits - 1)
            angle = random.uniform(0, 2*math.pi)
            code += f"qc.{gate}(math.pi/{int(2*math.pi/angle)}, q[{qubit}])\n"
        elif gate in ['cx', 'cz', 'swap']:
            control = random.randint(0, num_qubits - 1)
            target = random.randint(0, num_qubits - 1)
            while target == control:
                target = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control}], q[{target}])\n"
        elif gate in ['rzz', 'rxx']:
            qubit1 = random.randint(0, num_qubits - 1)
            qubit2 = random.randint(0, num_qubits - 1)
            while qubit2 == qubit1:
                qubit2 = random.randint(0, num_qubits - 1)
            angle = random.uniform(0, 2*math.pi)
            code += f"qc.{gate}(math.pi/{int(2*math.pi/angle)}, q[{qubit1}], q[{qubit2}])\n"
        elif gate == 'ccx':
            control1 = random.randint(0, num_qubits - 1)
            control2 = random.randint(0, num_qubits - 1)
            target = random.randint(0, num_qubits - 1)
            while control2 == control1 or target == control1 or target == control2:
                control2 = random.randint(0, num_qubits - 1)
                target = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control1}], q[{control2}], q[{target}])\n"
        elif gate == 'cswap':
            control = random.randint(0, num_qubits - 1)
            target1 = random.randint(0, num_qubits - 1)
            target2 = random.randint(0, num_qubits - 1)
            while target1 == control or target2 == control or target1 == target2:
                target1 = random.randint(0, num_qubits - 1)
                target2 = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control}], q[{target1}], q[{target2}])\n"
    
    with open(filename, 'w') as f:
        f.write(code)

### Generate random circuits in "Testcases" directory

In [3]:
os.makedirs("Testcases", exist_ok=True)
os.makedirs("Testcases\\deep", exist_ok=True)     # deep circuits
os.makedirs("Testcases\\shallow", exist_ok=True)  # shallow circuits

for i in range(10):
    generate_random_qiskit_circuit(3,25, f'Testcases\\deep\\deep_{i}.py')       #depth = 25
    generate_random_qiskit_circuit(3,5, f'Testcases\\shallow\\shallow_{i}.py')  # dpeth = 5

### Calculate Entropies without bugs and store in "Entropies" directory

In [ ]:
# for shallow
os.makedirs(r"Entropies\shallow", exist_ok=True)

testcases = r"Testcases\shallow"
config = r"muskit\executorConfig.py"
inputs = r"muskit\testcases.py"
savePath = r"Entropies\shallow\shallow.txt"

try:
    process = subprocess.Popen(
    [
        "python",
        "-m", "muskit.ComandMain",
        "Execute",
        config,
        inputs,
        testcases,
        savePath
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate()
    if stderr:
        print("Errors:", stderr)
except Exception as e:
    print(f"Error: {e}")

0

In [ ]:
# for deep

os.makedirs(r"Entropies\deep", exist_ok=True)
testcases = r"Testcases\deep"
config = r"muskit\executorConfig.py"
inputs = r"muskit\testcases.py"
savePath = r"Entropies\deep\deep.txt"

try:
    process = subprocess.Popen(
    [
        "python",
        "-m", "muskit.ComandMain",
        "Execute",
        config,
        inputs,
        testcases,
        savePath
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate()
    if stderr:
        print("Errors:", stderr)
except Exception as e:
    print(f"Error: {e}")

0

### Store bugless entropies in a list

In [12]:
# Read the content of the file
with open(r'Entropies\shallow\shallow.txt', 'r') as file:
    content = file.read().strip()

# Extract values and convert to float
shallow_entropies = [float(x) for x in content.split(',') if x]

with open(r'Entropies\deep\deep.txt', 'r') as file:
    content = file.read().strip()

# Extract values and convert to float
deep_entropies = [float(x) for x in content.split(',') if x]

print("Bugless shallow entropies: ",shallow_entropies)
print("Bugless deep entropies: ",deep_entropies)

Bugless shallow entropies:  [0.0, 0.0, 1.0102, 0.0, 0.0, 0.0, 1.7899, 0.3915, 0.0, 0.299]
Bugless deep entropies:  [1.5945, 2.3714, 2.8508, 2.9943, 0.9995, 2.88, 2.9805, 0.9983, 0.0, 2.416]


### Create mutants of testcases and Store in "Mutants" directory

In [ ]:
config = r"muskit\generatorConfig.py"

os.makedirs(r"Mutants\shallow", exist_ok=True)
testcases_dir = r"Testcases\shallow"
mutants_dir = r"Mutants\shallow"

for file in os.listdir(testcases_dir):
    if file.endswith(".py"):
        name = os.path.splitext(file)[0]
        try:
            process = subprocess.Popen([
            "python",
            "-m", "muskit.ComandMain",
            "Create",
            config,
            os.path.join(testcases_dir, file),
            os.path.join(mutants_dir, f"mutants_of_{name}")
            ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            stdout, stderr = process.communicate()
            if stderr:
                print("Errors:", stderr)
        except Exception as e:
            print(f"Error processing file {file}: {e}")


In [ ]:
os.makedirs(r"Mutants\deep", exist_ok=True)
testcases_dir = r"Testcases\deep"
mutants_dir = r"Mutants\deep"

for file in os.listdir(testcases_dir):
    if file.endswith(".py"):
        name = os.path.splitext(file)[0]
        try:
            process = subprocess.Popen([
            "python",
            "-m", "muskit.ComandMain",
            "Create",
            config,
            os.path.join(testcases_dir, file),
            os.path.join(mutants_dir, f"mutants_of_{name}")
            ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            stdout, stderr = process.communicate()
            if stderr:
                print("Errors:", stderr)
        except Exception as e:
            print(f"Error processing file {file}: {e}")

### Calculate entropies of mutants and Store in "Entropies" directory

In [ ]:
config = r"muskit\executorConfig.py"
inputs = r"muskit\testcases.py"

mutants_dir = r"Mutants\shallow"
for file in tqdm(os.listdir(mutants_dir), desc="Processing all mutants for each shallow circuit"):
    name = file[len("mutants_of_"):]
    savePath = rf"Entropies\shallow\{name}.txt"
    try:
        process = subprocess.Popen([
            "python",
            "-m", "muskit.ComandMain",
            "Execute",
            config,
            inputs,
            os.path.join(mutants_dir, file),
            savePath
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        if stderr:
            print("Errors:", stderr)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

Processing all mutants for each shallow circuit: 100%|██████████| 10/10 [55:29<00:00, 332.95s/it]


In [ ]:
mutants_dir = r"Mutants\deep"
for file in tqdm(os.listdir(mutants_dir), desc="Processing all mutants for each deep circuit"):
    name = file[len("mutants_of_"):]
    savePath = rf"Entropies\deep\{name}.txt"
    try:
        process = subprocess.Popen([
            "python",
            "-m", "muskit.ComandMain",
            "Execute",
            config,
            inputs,
            os.path.join(mutants_dir, file),
            savePath
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        if stderr:
            print("Errors:", stderr)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

Processing all mutants for each deep circuit: 100%|██████████| 10/10 [4:05:14<00:00, 1471.48s/it] 


### Plot probability ditribution curves on the Entropies of mutants for each testcase and Store in "Distributions" directory

In [ ]:
plt.rcParams.update({'font.size': 14})

In [22]:
figure_dir = r"Distributions\deep"
os.makedirs(figure_dir, exist_ok=True)

entropy_dir = r"Entropies\deep"
# List all files in the Entropies directory
entropy_files = os.listdir(entropy_dir)

# Iterate through each file and draw distribution charts
i=0
for file in entropy_files:
    if file.endswith('.txt') and file != "deep.txt":  # Process only .txt files, except the testcase entropies
        # Read the values from the file
        with open(os.path.join(entropy_dir, file), 'r') as f:
            # Split by comma and filter out empty strings before converting to float
            values = [float(x) for x in f.read().strip().split(',') if x]

        # Calculate mean and variance
        mean = np.mean(values)
        variance = np.var(values)

        # Create a smooth curve using KDE
        kde = stats.gaussian_kde(values)
        x_range = np.linspace(min(values), max(values), 1000)
        y_kde = kde(x_range)

        # Draw the distribution chart
        plt.figure(figsize=(10, 6))
        plt.plot(x_range, y_kde, 'b-', linewidth=2)
        plt.fill_between(x_range, y_kde, alpha=0.3)
        plt.axvline(mean, color='r', linestyle='--', label=f'Mean: {mean:.4f}')
        plt.axvline(mean + np.sqrt(variance), color='g', linestyle=':', label=f'Std Dev: {np.sqrt(variance):.4f}')
        plt.axvline(mean - np.sqrt(variance), color='g', linestyle=':')
        plt.axvline(deep_entropies[i], color='black', linestyle='-', label=f"Entropy without bugs: {deep_entropies[i]}")
        i+=1
        plt.title(f'Distribution of Entropies')
        plt.xlabel('Entropy Values')
        plt.ylabel('Density')
        plt.legend(fontsize=14, loc='upper left')
        plt.grid(True, alpha=0.3)

        # Save the chart as an image
        plt.savefig(os.path.join(figure_dir, f'distribution_{file[:-4]}.png'))
        plt.close()

In [23]:
figure_dir = r"Distributions\shallow"
os.makedirs(figure_dir, exist_ok=True)

entropy_dir = r"Entropies\shallow"
# List all files in the Entropies directory
entropy_files = os.listdir(entropy_dir)

# Iterate through each file and draw distribution charts
i=0
for file in entropy_files:
    if file.endswith('.txt') and file != "shallow.txt":  # Process only .txt files, except the testcase entropies
        # Read the values from the file
        with open(os.path.join(entropy_dir, file), 'r') as f:
            # Split by comma and filter out empty strings before converting to float
            values = [float(x) for x in f.read().strip().split(',') if x]

        # Calculate mean and variance
        mean = np.mean(values)
        variance = np.var(values)

        # Create a smooth curve using KDE
        kde = stats.gaussian_kde(values)
        x_range = np.linspace(min(values), max(values), 1000)
        y_kde = kde(x_range)

        # Draw the distribution chart
        plt.figure(figsize=(10, 6))
        plt.plot(x_range, y_kde, 'b-', linewidth=2)
        plt.fill_between(x_range, y_kde, alpha=0.3)
        plt.axvline(mean, color='r', linestyle='--', label=f'Mean: {mean:.4f}')
        plt.axvline(mean + np.sqrt(variance), color='g', linestyle=':', label=f'Std Dev: {np.sqrt(variance):.4f}')
        plt.axvline(mean - np.sqrt(variance), color='g', linestyle=':')
        plt.axvline(shallow_entropies[i], color='black', linestyle='-', label=f"Entropy without bugs: {shallow_entropies[i]}")
        i+=1
        plt.title(f'Distribution of Entropies')
        plt.xlabel('Entropy Values')
        plt.ylabel('Density')
        plt.legend(fontsize=14, loc='upper left')
        plt.grid(True, alpha=0.3)

        # Save the chart as an image
        plt.savefig(os.path.join(figure_dir, f'distribution_{file[:-4]}.png'))
        plt.close()

# Experiments with Quantum Algorithms

#### Function to convert a quantum circuit into a python file containing a circuit that is usable by muskit

In [ ]:
from qiskit import transpile

def quantum_circuit_to_py(qc, output_file):
    # Define allowed gates and their Qiskit names
    allowed_gates = {
        "x", "h", "p", "t", "s", "z", "y", "id",
        "rx", "ry", "rz", "sx", "swap", "rzz", 
        "rxx", "cx", "cz", "ccx", "cswap"
    }

    # Transpile circuit to use only allowed gates
    qc = transpile(
        qc,
        basis_gates=list(allowed_gates),
        optimization_level=3  # Aggressive optimization
    )

    with open(output_file, 'w') as f:
        f.write("import math\n")
        f.write("from qiskit import *\n\n")
        
        f.write(f"q = QuantumRegister({qc.num_qubits}, 'q')\n")
        f.write(f"c = ClassicalRegister({qc.num_clbits}, 'c')\n")
        f.write(f"qc = QuantumCircuit(q, c)\n\n")

        for instruction in qc.data:
            gate = instruction.operation
            qubits = instruction.qubits
            
            # Skip barrier and measurement operations
            if gate.name in ['barrier', 'measure']:
                continue
                
            # Handle parameterized gates
            if hasattr(gate, 'params') and gate.params:
                params = [f"math.pi/{int(math.pi/float(p))}" 
                         if p != 0 else "0" 
                         for p in gate.params]
                param_str = ", ".join(params)
            else:
                param_str = ""

            # Get qubit indices
            qubit_indices = [f"q[{qc.find_bit(q).index}]" for q in qubits]
            
            # Write gate operation
            if param_str:
                f.write(f"qc.{gate.name}({param_str}, {', '.join(qubit_indices)})\n")
            else:
                f.write(f"qc.{gate.name}({', '.join(qubit_indices)})\n")


#### Function to calculate entropy

In [ ]:
from math import log2
import numpy as np

def calculate_entropy(output_map):
    total_count = sum(output_map.values())

    entropy = 0
    for key, val in output_map.items():
        if (val == 0):
            continue
        else:
            p = val / total_count
            entropy = entropy - p * log2(p)

    return entropy

### Grover's algorithm

In [ ]:
# Built-in modules
import math

# Imports from Qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, errors
from qiskit.primitives import StatevectorSampler
from qiskit import transpile
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [ ]:
def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        
        qc.x(zero_inds)      # Line 1
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)    # Line 2
        qc.x(zero_inds)      # line 3
    return qc

def grover_search(marked_states, shots=100000, noise=None):
    oracle = grover_oracle(marked_states)
    grover_op = GroverOperator(oracle)

    optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
    )
    
    qc = QuantumCircuit(grover_op.num_qubits)
    # Create even superposition of all basis states
    qc.h(range(grover_op.num_qubits))
    # Apply Grover operator the optimal number of times
    qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
    # Measure all qubits
    qc.x(2)  #bug
    qc.measure_all()
    # qc.draw(output="mpl", style="iqp")
    
    sampler = AerSimulator(method='statevector', noise_model=noise)
    target = sampler.target
    pm = generate_preset_pass_manager(target=target, optimization_level=3)

    circuit_isa = pm.run(qc)
    # circuit_isa.draw(output="mpl", idle_wires=False, style="iqp")
    circuit_isa = circuit_isa.decompose(reps=10)
    result = sampler.run(circuit_isa, shots=shots, memory=True).result()
    return result.get_counts(), circuit_isa

In [7]:
result, circuit = grover_search(['000', '001'], shots=10000, noise=None)
print(result)

C:\Users\devra\AppData\Local\Temp\ipykernel_39748\69985639.py:28: DeprecationWarning: The class ``qiskit.circuit.library.generalized_gates.mcmt.MCMT`` is deprecated as of qiskit 1.4. It will be removed no earlier than 3 months after the release date. Use MCMTGate instead.
  qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)    # Line 2


{'101': 4937, '100': 5063}


#### Entropy without bugs

In [ ]:
grovers_entropy = calculate_entropy(result)
print(grovers_entropy)

0.9998854758372326


In [ ]:
os.makedirs("Algos", exist_ok=True)
quantum_circuit_to_py(circuit, r"Algos\grovers.py")  # circuit stored in Algos\grovers.py

Creating mutants of grover's circuit

In [ ]:
config = r"muskit\generatorConfig.py"

testcases_dir = r"Algos"
mutants_dir = r"Mutants\Algos"
os.makedirs(mutants_dir, exist_ok=True)

for file in os.listdir(testcases_dir):
    if file.endswith(".py"):
        name = os.path.splitext(file)[0]
        try:
            process = subprocess.Popen([
                "python",
                "-m", "muskit.ComandMain",
                "Create",
                config,
                os.path.join(testcases_dir, file),
                os.path.join(mutants_dir, f"mutants_of_{name}")
            ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            stdout, stderr = process.communicate()
            if stderr:
                print("Errors:", stderr)
        except Exception as e:
            print(f"Error processing file {file}: {e}")


Calculating entropies of grover's mutants

In [ ]:
config = r"muskit\executorConfig.py"
inputs = r"muskit\testcases.py"

os.makedirs(r"Entropies\Algos", exist_ok=True)

mutants_dir = r"Mutants\Algos"
for file in os.listdir(mutants_dir):
    name = file[len("mutants_of_"):]
    savePath = rf"Entropies\Algos\{name}.txt"
    try:
        process = subprocess.Popen([
            "python",
            "-m", "muskit.ComandMain",
            "Execute",
            config,
            inputs,
            os.path.join(mutants_dir, file),
            savePath
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        if stderr:
            print("Errors:", stderr)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

Storing the distributions in Distributions\Algos

In [ ]:
figure_dir = r"Distributions\Algos"
os.makedirs(figure_dir, exist_ok=True)

entropy_dir = r"Entropies\Algos"
# List all files in the Entropies directory
entropy_files = os.listdir(entropy_dir)

# Iterate through each file and draw distribution charts
i=0
for file in entropy_files:
    if file.endswith('.txt') and file != "deep.txt":  # Process only .txt files, except the testcase entropies
        # Read the values from the file
        with open(os.path.join(entropy_dir, file), 'r') as f:
            # Split by comma and filter out empty strings before converting to float
            values = [float(x) for x in f.read().strip().split(',') if x]

        # Calculate mean and variance
        mean = np.mean(values)
        variance = np.var(values)

        # Create a smooth curve using KDE
        kde = stats.gaussian_kde(values)
        x_range = np.linspace(min(values), max(values), 1000)
        y_kde = kde(x_range)

        # Draw the distribution chart
        plt.figure(figsize=(10, 6))
        plt.plot(x_range, y_kde, 'b-', linewidth=2)
        plt.fill_between(x_range, y_kde, alpha=0.3)
        plt.axvline(mean, color='r', linestyle='--', label=f'Mean: {mean:.4f}')
        plt.axvline(mean + np.sqrt(variance), color='g', linestyle=':', label=f'Std Dev: {np.sqrt(variance):.4f}')
        plt.axvline(mean - np.sqrt(variance), color='g', linestyle=':')
        plt.axvline(grovers_entropy, color='black', linestyle='-', label=f"Entropy without bugs: {entropy:.4f}")
        i+=1
        plt.title(f'Distribution of Entropies')
        plt.xlabel('Entropy Values')
        plt.ylabel('Density')
        plt.legend(fontsize=15)
        plt.grid(True, alpha=0.3)

        # Save the chart as an image
        plt.savefig(os.path.join(figure_dir, f'distribution_{file[:-4]}.png'))
        plt.close()